# Bag of Word + SVM

## Dataset

The dataset is Reuters-21578 dataset - news documents collected from the Reuter’s newswire in 1987. There are 7310 training documents, 3355 test documents. Each document contains text and labels. For example

**Text:**
"chicago, march 11 - u.s. economic data this week could be the key in determining whether u.s. interest rate futures break out of a 3-1/2 month trading range, financial analysts said ...:"

**Labels:**
castor seed,potato,barley,soybean,gas,crude,nickel,coconut,nkr,platinum,citrus pulp,yen,cotton,dfl,copper,fishmeal,dmk,hog,jobs,lead,rubber,interest,corn gluten feed,cruzado,inventories,grain,sugar,oat,ship,palm kernel,alum,reserves,...

In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
result_path = 'running_result/tfidf-svm'

# Data-preprocess part. 

Construct documents as $[d_1,d_2,...,d_m]$ , where $d_i$ is string. 

Construct labels as $[y_1,y_2,...,y_m]$, where $y_i$ is variable-length list of string $[l_1,...,l_k]$.

In [4]:
from src.data_preprocess.preprocess import DataSet
dataset = DataSet()
raw_x_train, raw_y_train, raw_x_test, raw_y_test = dataset.get_train_and_test_documents()

========== Extract data from pickle files ==========


Convert multilabel to single label

In [5]:
x_train_single_raw = []
y_train_single_raw = []

class_df = {}

for i, label_list in enumerate(raw_y_train):
    for label in set(label_list):
        x_train_single_raw.append(raw_x_train[i])
        y_train_single_raw.append(label)

for label in y_train_single_raw:
    class_df[label] = class_df.get(label, 0) + 1

x_train_single = []
y_train = []

for i, x in enumerate(x_train_single_raw):
    if class_df[y_train_single_raw[i]] >= 5:
        x_train_single.append(x)
        y_train.append(y_train_single_raw[i])

In [6]:
len(x_train_single), len(raw_x_train), len(y_train)

(8547, 7310, 8547)

## Preprocess text

1. Tokenize document text into a list of tokens \['u.s.', 'econom', 'data', 'debt', 'chicago', 'interest',...\] using NLTK library and remove stop-words from tokens, such as \['a', 'the', ...\]. 

In [7]:
from nltk.tokenize import word_tokenize
from src.data_preprocess.preprocess import stop_words_
from nltk.stem.porter import PorterStemmer
import re

stem = PorterStemmer()

def convert_text_to_word_list(text: str) -> []:
    word_tokens = word_tokenize(text)
    tokens = []
    for word in word_tokens:
        # remove punctturation and stop word
        word = word.replace('*', '')
        if re.compile(r'[a-z]').search(word) is None or word in stop_words_ or len(word) <= 3:
            continue
        word = stem.stem(word)
        if len(word) <= 3:
            continue
        tokens.append(word)
    return tokens

In [8]:
x_train_word_list = [convert_text_to_word_list(text) for text in x_train_single]
x_test_word_list = [convert_text_to_word_list(text) for text in raw_x_test]

2. Collect all tokens as a set of words - vocabulary.

In [9]:
words = set()
for word_list in x_train_word_list:
    words = words.union(word_list)

In [10]:
len(words)

19120

# Feature engineering.

We use vocabulary V to build feature vector for each document.

Vocabulary = {'econom', 'data', 'year', …}

Document = \['econom', 'interest', 'date', …\]  -> \[1, 0, 0, …\] -- (|V|, )

|   T      |  'econom'  |  'data' | 'year' | ... |
|:--------:|:----------:|:-------:|:------:|:---:|
| document |      1     |     0   |    0   | ... |

One-hot method will convert each document to a binary vector. Combining with tf-idf as weight for each term, we can map text to a vector f ~ (|V|, ) and then convert list of documents \[d1,d2,...,dm\] to a matrix X ~ (m, |V|).

1. calculate document frequency of each term

In [11]:
df = {}
for word_list in x_train_word_list:
    for word in set(word_list):
        df[word] = df.get(word, 0) + 1

2. map word list to a vector using tf-idf 

In [12]:
def calculate_tf_idf(tf, df, doc_num):
    """
    :param doc_num: the number of all documents
    :param tf: term frequency
    :param df: document frequency where term appears
    :return: td-idf importance
    """
    idf = np.log(float(doc_num + 1) / (df + 1))
    tf = 1.0 + np.log(tf)
    return tf * idf

In [13]:
import numpy as np
n = len(x_train_word_list)

def map_word_list_to_tfidf_vector(word_list, words):
    vocabulary = {}
    for i, word in enumerate(words):
        vocabulary[word] = i
    
    x = np.zeros(len(vocabulary))
    tf = {}
    
    for word in word_list:
        if word in vocabulary.keys():
            tf[word] = tf.get(word, 0) + 1
    
    for word in tf.keys():
        j = vocabulary[word]
        x[j] = calculate_tf_idf(tf[word], df[word], n)

    return x.tolist()

In [14]:
x_train = [map_word_list_to_tfidf_vector(word_list, words) for word_list in x_train_word_list]
x_test = [map_word_list_to_tfidf_vector(word_list, words) for word_list in x_test_word_list]

# Classification

Input is X(m, |V|), labels is Y(m,). Use SVM to solve the classification problem.

## Model Selection

We use K-fold cross validation to select best model between different models which may have different parameters and different kernels (linear, RBF). For example

```python
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100]}, {'kernel': ['linear'], 'C': [1, 10, 100]}]
```


In [15]:
# cross_validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5], 'C': [1, 10, 100]}, {'kernel': ['linear'], 'C': [1, 10, 100]}]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC
import time
import pandas as pd

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# train svm via cross validation
A = time.time()
clf = GridSearchCV(SVC(), tuned_parameters, cv=5, scoring='precision_weighted', verbose=15, n_jobs=-1)
clf.fit(x_train, y_train)
print('Cross validation time: {}'.format(time.time() - A))
print("Best parameters set found on development set:")
print(clf.best_params_)

cv_result = pd.DataFrame.from_dict(clf.cv_results_)
with open('{}/cv_result.csv'.format(result_path),'w') as f:
    cv_result.to_csv(f)

# save model
joblib.dump(clf, '{}/cv_model.joblib'.format(result_path))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 120.9min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 121.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 123.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 183.9min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 187.6min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 188.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 189.0min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 189.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 237.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 239.9min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 239.9min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 298.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 303.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 303.7min
[Parallel(n_jobs=-1)]: Done  15 tasks     

Finally we found the best parameters as following:

```python
Best parameters set found on development set:
{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.374 (+/-0.046) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.193 (+/-0.024) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.446 (+/-0.053) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.437 (+/-0.073) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.424 (+/-0.059) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.447 (+/-0.035) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.409 (+/-0.036) for {'C': 1, 'kernel': 'linear'}
0.409 (+/-0.038) for {'C': 10, 'kernel': 'linear'}
0.409 (+/-0.038) for {'C': 100, 'kernel': 'linear'}
```

We use the accuracy as the performance metric of our model.

In [ ]:
def cal_predict_accuracy(y_pred, y_test):
    # test
    sum_acc = 0
    for j, pre in enumerate(y_pred):
        if pre in y_test[j]:
           sum_acc += 1

    return sum_acc / len(y_pred)

In [ ]:
# test
y_pred = clf.predict(x_test)
print("Accuracy: {}".format(cal_predict_accuracy(y_pred=y_pred, y_test=raw_y_test)))

# Effect of feature selection

The vocabulary size is about 20000. It is useful to sort the vocabulary by the importance of each word which can be calculated by total term frequency, chi square and tf-idf for class, because the vocabulary size N equals the dimension of document feature vector and larger N will cause high time complexity, so we can select top N words instead of selecting all terms for reducing time complexity of training.

Select tf-idf as importance metric for terms.

In [ ]:
import sys
from tqdm import tqdm
    
def calculate_priority_by_tfidf(X: [], y: []) -> {}:
    term_df = {}
    term_tf = {}
    tf = {}
    min_count = 4
    _n = len(X)
    i = 0
    
    for word_list in tqdm(X):
        terms = set(word_list)
        for term in terms:
            term_df[term] = term_df.get(term, 0) + 1

        class_ = y[i]
        i += 1
        if class_ not in term_tf.keys():
            term_tf[class_] = {}

        for term in word_list:
            term_tf[class_][term] = term_tf[class_].get(term, 0) + 1

    for class_ in term_tf.keys():
        for term, v in term_tf[class_].items():
            if v >= min_count:
                tf[term] = tf.get(term, 0) + v

    term_importance_pair = {}
    
    for term in tf.keys():
        term_importance_pair[term] = calculate_tf_idf(tf[term], term_df[term], _n)

    return term_importance_pair

In [ ]:
def generate_bag_of_words(calculate_priority=calculate_priority_by_tfidf) -> []:
    n_doc = len(x_train_word_list)

    #print("========== Feature selection ==========")
    term_importance_pair = calculate_priority(x_train_word_list, y_train)
    temp = []
    for term, value in term_importance_pair.items():
        temp.append([term, value])

    #print("Sort terms by importance...")
    temp = sorted(temp, key=lambda pair: pair[1], reverse=True)
    vocabulary = [item[0] for item in temp]

    if len(vocabulary) == 0:
        print("Warning! After pruning, no terms remain. Try a lower min_df or a higher max_df.")

    return vocabulary

In [ ]:
sorted_vocabulary = generate_bag_of_words(calculate_priority_by_tfidf)

We can use the best parameters from the cross validation part.

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', gamma=0.0001, C=100)

In [ ]:
import time

accuracy = []
vocab_size_step = [int(i) for i in np.logspace(start=np.log10(100), stop=np.log10(len(vocabulary)), num=10, base=10)]

for vocab_size in vocab_size_step:
    A = time.time()
    tmp_vocab = sorted_vocabulary[0:vocab_size]
    print('vocabulary_size: {}'.format(vocab_size))

    x_train_tmp = [map_word_list_to_tfidf_vector(word_list, tmp_vocab) for word_list in x_train_word_list]
    x_test_tmp = [map_word_list_to_tfidf_vector(word_list, tmp_vocab) for word_list in x_test_word_list]
    
    svm_model.fit(x_train_tmp, y_train)
    y_pred = svm_model.predict(x_test_tmp)
    acc = cal_predict_accuracy(y_pred=y_pred, y_test=raw_y_test)
    print("Accuracy: {}, Time: {}".format(acc, time.time() - A))
    accuracy.append(acc)

plt.plot(vocab_size_step, accuracy, marker='x')
plt.xlabel('vocabulary_size')
plt.ylabel('accuracy')
plt.title('classification accuracy with respect to different vocabulary_size')
plt.savefig('{}/acc-ver-vocab-size.svg'.format(result_path))
with open('{}/acc-vocab-size.txt'.format(result_path), 'w') as f:
    f.writelines('{}'.format(vocab_size_step))
    f.writelines('{}'.format(accuracy))

Finally, the best result is 0.89 accuracy.However, when vocabulary size is 8000, the accuracy (0.82) is good enough which means feature selection can benefit on reducing time complexity as well as keep performance of classification.

![](https://i.loli.net/2018/12/29/5c2708e4d3b05.jpg)